# Demo pipeline for generating mock galaxy samples with LSST-like photometry

author: Luca Tortorelli

last run successfully: Oct 19, 2023

This notebook demonstrates some basic usage of the `rail_lib_gp_comp` package. We use this package and others implemented in RAIL to create a small pipeline that:
- Samples galaxy physical properties from the diffsky/skysim galaxy population model.
- Generates rest-frame SEDs with rail_dsps for each galaxy based on their sampled physical properties.
- Computes model apparent magnitudes for the LSST bands from the rest-frame SEDs.
- Degrades the model apparent magnitudes into LSST-like observed apparent magnitudes using the LSSTErrorModel.

This pipeline allows the user to obtain catalogs of galaxies with physical properties, mock LSST-like photometry and rest-frame SEDs.

#### Generating the population parameters from the diffsky/skysim model

The `rail_lib_gp_comp` package contains in the `examples_data` folder a diffsky/skysim v3.1.0 input catalog in parquet format constituted of 10k sources selected having LSST_obs_i<=24.1 (LSST Y1 gold sample cut). Generating the population parameters with the class `DiffskyGalaxyPopulationModeler` is equivalent, in this case, from reading the parameters necessary to the `diffstar` and `diffmah` packages to create galaxy star-formation histories.

For other galaxy population models, the `GalaxyPopulationModeler` class will generate the population parameters from e.g. a prior distribution.

The output of this piece of the pipeline is an hdf5 table containing the population parameters, as well as redshifts, metallicity means and scatters. For other galaxy population models, this would be an hdf5 table containing only population parameters drawn from a prior.

In [ ]:
import os
from rail.creation.engines.galaxy_population_components_modeler import DiffskyGalaxyPopulationModeler
import rail.lib_gp_comp
from rail.core.stage import RailStage
from rail.core.data import TableHandle

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
RAIL_LIB_GP_COMP_DIR = os.path.abspath(os.path.join(os.path.dirname(rail.lib_gp_comp.__file__), '..'))
default_files_folder = os.path.join(RAIL_LIB_GP_COMP_DIR, 'examples_data', 'creation_data', 'data')
trainFile = os.path.join(default_files_folder, 'skysim_v3.1.0_10k_lssty1cut.pq')

In [ ]:
training_data = DS.read_file("training_data", TableHandle, trainFile)

In [ ]:
diffskygalaxypopulationmodeler = DiffskyGalaxyPopulationModeler.make_stage(name='DiffskyGalaxyPopulationModeler',
                                                                 diffmah_keys=["diffmah_logmp_fit", "diffmah_mah_logtc",
                                                                               "diffmah_early_index", "diffmah_late_index"],
                                                                 diffstar_ms_keys=['diffstar_u_lgmcrit', 'diffstar_u_lgy_at_mcrit',
                                                                                   'diffstar_u_indx_lo', 'diffstar_u_indx_hi',
                                                                                   'diffstar_u_tau_dep'],
                                                                 diffstar_q_keys=['diffstar_u_qt', 'diffstar_u_qs', 'diffstar_u_q_drop',
                                                                                  'diffstar_u_q_rejuv'],
                                                                 catalog_redshift_key='redshift',
                                                                 catalog_metallicity_key='lg_met_mean',
                                                                 catalog_metallicity_scatter_key='lg_met_scatter')

In [ ]:
diffskygalaxypopulationmodel = diffskygalaxypopulationmodeler.fit_model(input_data=training_data)

In [ ]:
diffskygalaxypopulationmodel.data

#### Sampling galaxy properties from the population parameters

This class creates an input catalog of galaxy physical properties using the population parameters sampled from the prior. For instance, if we sample luminosity function M* and phi*, then this class would return the actual galaxy luminosities and redshifts from a luminosity function having parameters drawn from the prior.

In this case, the population parameters drawn from the diffsky/skysim model are used by `diffmah` and `diffstar` to generate galaxy star-formation and stellar mass histories, as well as stellar mass and star-formation rates at the time of observation. These properties are stored together with redshifts, stellar metallicity means and scatters into an hdf5 table.

In [ ]:
from rail.creation.engines.galaxy_population_components_creator import DiffskyGalaxyPopulationCreator
from diffstar.defaults import DEFAULT_N_STEPS, LGT0, FB, T_BIRTH_MIN
from dsps.constants import T_TABLE_MIN
from dsps.cosmology import DEFAULT_COSMOLOGY
from rail.core.data import Hdf5Handle

In [ ]:
trainFile = 'model_DiffskyGalaxyPopulationModeler.hdf5'

In [ ]:
training_data = DS.read_file("training_data", Hdf5Handle, trainFile)

In [ ]:
diffskygalaxypopulationcreator = DiffskyGalaxyPopulationCreator.make_stage(name='DiffskyGalaxyPopulationCreator',
                                                                           log10_age_universe=LGT0,
                                                                           cosmic_baryon_fraction=FB,
                                                                           t_min_table=T_TABLE_MIN,
                                                                           t_max_table=10 ** LGT0,
                                                                           n_time_steps=DEFAULT_N_STEPS,
                                                                           tacc_integration_min=T_BIRTH_MIN,
                                                                           cosmology_parameters=DEFAULT_COSMOLOGY,
                                                                           catalog_redshift_key='redshift',
                                                                           catalog_metallicity_key='lg_met_mean',
                                                                           catalog_metallicity_scatter_key='lg_met_scatter',
                                                                           cosmic_time_grid_key='cosmic_time_grid',
                                                                           star_formation_history_key='star_formation_history',
                                                                           star_formation_rate_key='star_formation_rate',
                                                                           stellar_mass_history_key='stellar_mass_history',
                                                                           stellar_mass_key='stellar_mass')

In [ ]:
diffskygalaxypopulationproperties = diffskygalaxypopulationcreator.sample(input_data=training_data)

In [ ]:
diffskygalaxypopulationproperties.data

#### Generating rest-frame SEDs from galaxy properties using rail_dsps

The two previous piece of the pipeline constitute the input data for `rail_dsps`. This package is an interface in RAIL to DSPS (Hearin+22). In particular, the star-formation histories, the redshifts and the stellar metallicities are used to generate the corresponding rest-frame SED of each galaxy at the time of observation.

The output is again an hdf5 table storing the rest-frame SEDs and the galaxy redshifts.

In [ ]:
from rail.creation.engines.dsps_sed_modeler import DSPSSingleSedModeler

In [ ]:
trainFile = 'output_DiffskyGalaxyPopulationCreator.hdf5'
training_data = DS.read_file("training_data", TableHandle, trainFile)

In [ ]:
dspssinglesedmodeler = DSPSSingleSedModeler.make_stage(name='DSPSSingleSedModeler', 
                                                       ssp_templates_file='ssp_data_fsps_v3.2_lgmet_age.h5',
                                                       redshift_key='redshift',
                                                       cosmic_time_grid_key='cosmic_time_grid',
                                                       star_formation_history_key='star_formation_history',
                                                       stellar_metallicity_key='lg_met_mean',
                                                       stellar_metallicity_scatter_key='lg_met_scatter')

In [ ]:
dspssinglesedmodel = dspssinglesedmodeler.fit_model(input_data=training_data)

In [ ]:
dspssinglesedmodel.data

#### Computing model apparent magnitudes in the LSST bands

Using `rail_dsps` and the generated rest-frame SEDs, we can compute the model apparent magnitudes and absolute magnitudes for the LSST filters. The output is stored into an hdf5 table.

In [ ]:
import rail.dsps
from rail.creation.engines.dsps_photometry_creator import DSPSPhotometryCreator

In [ ]:
trainFile = 'model_DSPSSingleSedModeler.hdf5'

In [ ]:
training_data = DS.read_file("training_data", TableHandle, trainFile)

In [ ]:
RAIL_DSPS_DIR = os.path.abspath(os.path.join(os.path.dirname(rail.dsps.__file__), '..'))
default_files_folder = os.path.join(RAIL_DSPS_DIR, 
                                    'examples_data/creation_data/data/dsps_default_data')

In [ ]:
dspsphotometrycreator = DSPSPhotometryCreator.make_stage(name='DSPSPhotometryCreator',
                                                         redshift_key='redshift',
                                                         restframe_sed_key='restframe_sed',
                                                         absolute_mags_key='rest_frame_absolute_mags',
                                                         apparent_mags_key='apparent_mags',
                                                         filter_folder=os.path.join(default_files_folder, 'filters'),
                                                         instrument_name='lsst',
                                                         wavebands='u,g,r,i,z,y',
                                                         ssp_templates_file=os.path.join(default_files_folder,'ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                         default_cosmology=True)

In [ ]:
dspsphotometry = dspsphotometrycreator.sample(input_data=training_data)

In [ ]:
dspsphotometry.data

#### Degrading magnitudes to observed LSST-like apparent magnitudes

The final step of the pipeline involves the use of the `ObsCondition` and `LSSTErrorModel` classes implemented in `rail_astro_tools`. These classes allow us to perform the mapping from model to observed magnitudes, i.e. applying a transfer function. The output of this process is a catalog in parquet format containing LSST observed magnitudes and errors having the same consecutive indices of the input model magnitudes.

In [ ]:
from rail.creation.degradation.observing_condition_degrader import ObsCondition
import numpy as np
import pandas as pd

In [ ]:
lsst_z_mags = np.empty((len(dspsphotometry.data['apparent_mags']), 7))
lsst_z_mags[:, 0] = dspssinglesedmodel.data['redshift']
lsst_z_mags[:, 1] = dspsphotometry.data['apparent_mags'][:,0]
lsst_z_mags[:, 2] = dspsphotometry.data['apparent_mags'][:,1]
lsst_z_mags[:, 3] = dspsphotometry.data['apparent_mags'][:,2]
lsst_z_mags[:, 4] = dspsphotometry.data['apparent_mags'][:,3]
lsst_z_mags[:, 5] = dspsphotometry.data['apparent_mags'][:,4]
lsst_z_mags[:, 6] = dspsphotometry.data['apparent_mags'][:,5]

In [ ]:
input_redshift_mags = pd.DataFrame(lsst_z_mags, columns=["redshift", "u", "g", "r", "i", "z", "y"])
data = DS.add_data("data", input_redshift_mags, TableHandle, path="dsps_lsst_model_mags.pd")

In [ ]:
degrader = ObsCondition.make_stage()
degraded_data = degrader(data).data

In [ ]:
degraded_data